In [2]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

/ldap_home/qishuai.zhong/miniconda3/envs/punctuator_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset conll2003 (/ldap_home/qishuai.zhong/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 629.68it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]

print(train_dataset, validation_dataset)

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
}) Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 3250
})


In [5]:
# sample 5000 for training

import random

index_list = [index for index, _ in enumerate(train_dataset["tokens"])]
random.seed(7)

sample_indexes = random.sample(index_list, 5000)
print(sample_indexes[:10])

training_corpus = [sequence for index, sequence in enumerate(train_dataset["tokens"]) if index in sample_indexes]
training_tags = [tags for index, tags in enumerate(train_dataset["ner_tags"]) if index in sample_indexes]

print(training_corpus[:5], [len(sequence) for sequence in training_corpus[:5]],  training_tags[:5], [len(sequence) for sequence in training_tags[:5]])


[5305, 2471, 6468, 10664, 791, 1186, 13455, 8779, 1542, 5991]
[['BRUSSELS', '1996-08-22'], ['The', 'European', 'Commission', 'said', 'on', 'Thursday', 'it', 'disagreed', 'with', 'German', 'advice', 'to', 'consumers', 'to', 'shun', 'British', 'lamb', 'until', 'scientists', 'determine', 'whether', 'mad', 'cow', 'disease', 'can', 'be', 'transmitted', 'to', 'sheep', '.'], ['"', 'We', 'do', "n't", 'support', 'any', 'such', 'recommendation', 'because', 'we', 'do', "n't", 'see', 'any', 'grounds', 'for', 'it', ',', '"', 'the', 'Commission', "'s", 'chief', 'spokesman', 'Nikolaus', 'van', 'der', 'Pas', 'told', 'a', 'news', 'briefing', '.'], ['He', 'said', 'further', 'scientific', 'study', 'was', 'required', 'and', 'if', 'it', 'was', 'found', 'that', 'action', 'was', 'needed', 'it', 'should', 'be', 'taken', 'by', 'the', 'European', 'Union', '.'], ['The', 'EU', "'s", 'scientific', 'veterinary', 'and', 'multidisciplinary', 'committees', 'are', 'due', 'to', 're-examine', 'the', 'issue', 'early', 'ne

In [6]:
# get the longest sentence
 
lengths = [len(text) for text in training_corpus]
print(max(lengths))

113


In [ ]:
from dbpunctuator.training import NERTrainingArguments, NERTrainingPipeline
from datasets import load_dataset

training_args = NERTrainingArguments(
    training_corpus = training_corpus,
    validation_corpus = validation_dataset["tokens"],
    training_tags = training_tags,
    validation_tags = validation_dataset["ner_tags"],
    model_name_or_path="distilbert-base-cased",
    tokenizer_name="distilbert-base-cased",
    epoch=20,
    batch_size=32,
    model_storage_dir="../models/corll_ner_no_rdrop_sampled",
    addtional_model_config={"dropout": 0.3, "attention_dropout": 0.3},
    gpu_device=1,
    warm_up_steps=500,
    r_drop=False,
    tensorboard_log_dir="../runs/corll_ner_no_rdrop_sampled",
    label2id={
        "O": 0,
        "B-PER": 1,
        "I-PER": 2,
        "B-ORG": 3,
        "I-ORG": 4,
        "B-LOC": 5,
        "I-LOC": 6,
        "B-MISC": 7,
        "I-MISC": 8,
    }
)

training_pipeline = NERTrainingPipeline(training_args)
training_pipeline.run()

In [ ]:
from dbpunctuator.training import NERTrainingArguments, NERTrainingPipeline
from datasets import load_dataset

training_args = NERTrainingArguments(
    training_corpus = train_dataset["tokens"],
    validation_corpus = validation_dataset["tokens"],
    training_tags = train_dataset["ner_tags"],
    validation_tags = validation_dataset["ner_tags"],
    model_name_or_path="distilbert-base-cased",
    tokenizer_name="distilbert-base-cased",
    epoch=20,
    batch_size=32,
    model_storage_dir="../models/corll_ner_rdrop",
    addtional_model_config={"dropout": 0.3, "attention_dropout": 0.3},
    gpu_device=1,
    warm_up_steps=500,
    r_drop=True,
    tensorboard_log_dir="../runs/corll_ner_rdrop",
    label2id={
        "O": 0,
        "B-PER": 1,
        "I-PER": 2,
        "B-ORG": 3,
        "I-ORG": 4,
        "B-LOC": 5,
        "I-LOC": 6,
        "B-MISC": 7,
        "I-MISC": 8,
    }
)

training_pipeline = NERTrainingPipeline(training_args)
training_pipeline.run()

In [9]:
test_dataset = dataset["test"]

In [13]:
from dbpunctuator.training import EvaluationArguments, EvaluationPipeline

validation_args = EvaluationArguments(
    evaluation_corpus=test_dataset["tokens"],
    evaluation_tags=test_dataset["ner_tags"],
    model_name_or_path="../models/corll_ner_no_rdrop_sampled",
    tokenizer_name="distilbert-base-cased",
    batch_size=32,
    gpu_device=1,
)

validate_pipeline = EvaluationPipeline(validation_args)
validate_pipeline.run()

2022-09-10 23:43:02,499 - INFO - evalute.py:66 - evalute.tokenize - 44779 - tokenize data
2022-09-10 23:43:10,577 - INFO - evalute.py:138 - evalute._encode_tags - 44779 - encoding tags
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3453/3453 [00:00<00:00, 9346.95it/s]
2022-09-10 23:43:10,955 - INFO - evalute.py:82 - evalute.validate - 44779 - start validation
Processing batch: 108: 100%|██████████████████████████████████████████████████████████████████████████████| 108/108 [00:05<00:00, 21.34it/s, Last_batch_loss=0.13]
2022-09-10 23:43:16,071 - INFO - evalute.py:132 - evalute.validate - 44779 - validation report: 
               precision    recall  f1-score   support

       B-LOC     0.8831    0.9329    0.9073      1668
      B-MISC     0.7849    0.8162    0.8003       702
       B-ORG     0.8830    0.8724    0.8776      1661
       B-PER     0.9540    0.9487    0.9513      1617
       I-LOC     0.7389    

In [11]:
from dbpunctuator.training import EvaluationArguments, EvaluationPipeline

validation_args = EvaluationArguments(
    evaluation_corpus=test_dataset["tokens"],
    evaluation_tags=test_dataset["ner_tags"],
    model_name_or_path="../models/corll_ner_rdrop_sampled",
    tokenizer_name="distilbert-base-cased",
    batch_size=32,
    gpu_device=1,
)

validate_pipeline = EvaluationPipeline(validation_args)
validate_pipeline.run()

2022-09-10 23:12:35,885 - INFO - evalute.py:66 - evalute.tokenize - 44779 - tokenize data
2022-09-10 23:12:44,473 - INFO - evalute.py:138 - evalute._encode_tags - 44779 - encoding tags
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3453/3453 [00:00<00:00, 9495.29it/s]
2022-09-10 23:12:44,845 - INFO - evalute.py:82 - evalute.validate - 44779 - start validation
Processing batch: 108: 100%|██████████████████████████████████████████████████████████████████████████████| 108/108 [00:04<00:00, 21.64it/s, Last_batch_loss=0.24]
2022-09-10 23:12:49,895 - INFO - evalute.py:132 - evalute.validate - 44779 - validation report: 
               precision    recall  f1-score   support

       B-LOC     0.9278    0.9017    0.9146      1668
      B-MISC     0.7334    0.8348    0.7808       702
       B-ORG     0.8659    0.8826    0.8742      1661
       B-PER     0.9529    0.9511    0.9520      1617
       I-LOC     0.7835    

In [15]:
from dbpunctuator.training import EvaluationArguments, EvaluationPipeline

validation_args = EvaluationArguments(
    evaluation_corpus=test_dataset["tokens"],
    evaluation_tags=test_dataset["ner_tags"],
    model_name_or_path="../models/corll_ner_no_rdrop",
    tokenizer_name="distilbert-base-cased",
    batch_size=32,
    gpu_device=1,
)

validate_pipeline = EvaluationPipeline(validation_args)
validate_pipeline.run()

2022-09-11 00:13:46,102 - INFO - evalute.py:66 - evalute.tokenize - 44779 - tokenize data
2022-09-11 00:13:54,662 - INFO - evalute.py:138 - evalute._encode_tags - 44779 - encoding tags
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3453/3453 [00:00<00:00, 11301.89it/s]
2022-09-11 00:13:54,976 - INFO - evalute.py:82 - evalute.validate - 44779 - start validation
Processing batch: 108: 100%|██████████████████████████████████████████████████████████████████████████████| 108/108 [00:05<00:00, 21.51it/s, Last_batch_loss=0.04]
2022-09-11 00:14:00,058 - INFO - evalute.py:132 - evalute.validate - 44779 - validation report: 
               precision    recall  f1-score   support

       B-LOC     0.9243    0.9371    0.9306      1668
      B-MISC     0.8039    0.8234    0.8135       702
       B-ORG     0.8792    0.9031    0.8910      1661
       B-PER     0.9648    0.9505    0.9576      1617
       I-LOC     0.8417    

In [16]:
from dbpunctuator.training import EvaluationArguments, EvaluationPipeline

validation_args = EvaluationArguments(
    evaluation_corpus=test_dataset["tokens"],
    evaluation_tags=test_dataset["ner_tags"],
    model_name_or_path="../models/corll_ner_rdrop",
    tokenizer_name="distilbert-base-cased",
    batch_size=32,
    gpu_device=1,
)

validate_pipeline = EvaluationPipeline(validation_args)
validate_pipeline.run()

2022-09-11 00:14:13,330 - INFO - evalute.py:66 - evalute.tokenize - 44779 - tokenize data
2022-09-11 00:14:20,863 - INFO - evalute.py:138 - evalute._encode_tags - 44779 - encoding tags
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3453/3453 [00:00<00:00, 12193.31it/s]
2022-09-11 00:14:21,154 - INFO - evalute.py:82 - evalute.validate - 44779 - start validation
Processing batch: 108: 100%|██████████████████████████████████████████████████████████████████████████████| 108/108 [00:05<00:00, 21.28it/s, Last_batch_loss=0.21]
2022-09-11 00:14:26,287 - INFO - evalute.py:132 - evalute.validate - 44779 - validation report: 
               precision    recall  f1-score   support

       B-LOC     0.9273    0.9323    0.9297      1668
      B-MISC     0.7931    0.8462    0.8187       702
       B-ORG     0.9106    0.8832    0.8967      1661
       B-PER     0.9404    0.9666    0.9533      1617
       I-LOC     0.7940    